In [1]:
import pandas

from util import ESConnection
from elasticsearch_dsl import Search

es_conn = ESConnection()

In [2]:
s = Search(using=es_conn, index='git')

# Unique count of Commits by Project (max 100 projects)
s.aggs.bucket('authors', 'terms', field='author_name', size=20000)\
    .metric('commits', 'cardinality', field='hash')
result = s.execute()

# In case you need to check response, uncomment line below
#print(result.to_dict()['aggregations'])


In [11]:
df = pandas.DataFrame()

df = df.from_dict(result.to_dict()['aggregations']['authors']['buckets'])
df = df.drop('doc_count', axis=1)
df['commits'] = df['commits'].apply(lambda row: row['value'])
df=df[['key', 'commits']]
df.columns = ['author', 'commits']

In [12]:
df

,author,commits
0,Hallvord Reiar Michaelsen Steen,220679
1,B2G Bumper Bot,50308
2,GitHub,71182
3,Ryan VanderMeulen,19142
4,Cameron Dawson,23731
5,Tomcat,13795
6,Diego Marcos Segura,14782
7,Robert O'Callahan,14733
8,KWierso,10374
9,David Baron,12059


In [13]:
from enrich import Gender

### 'xxx' string is used here to hide gendereize key
gender = Gender(df, "xxx")
test = gender.enrich("author")

In [14]:
test

,author,commits,gender_analyzed_name,gender_probability,gender,gender_count
0,Hallvord Reiar Michaelsen Steen,220679,Hallvord,0.00,NotKnown,0
1,B2G Bumper Bot,50308,B2G,0.00,NotKnown,0
2,GitHub,71182,GitHub,0.00,NotKnown,0
3,Ryan VanderMeulen,19142,Ryan,0.99,male,4755
4,Cameron Dawson,23731,Cameron,0.94,male,614
5,Tomcat,13795,Tomcat,1.00,male,1
6,Diego Marcos Segura,14782,Diego,1.00,male,2270
7,Robert O'Callahan,14733,Robert,1.00,male,5222
8,KWierso,10374,KWierso,0.00,NotKnown,0
9,David Baron,12059,David,1.00,male,12593


## Contributors by Gender

Number of people by gender is shown below. We use [Genderize API](https://genderize.io/) to infer gender based on first name. For those cases in which gender can't be identified, 'NotKnown' string is used.

In [15]:
copy_test = test.copy()

In [17]:
gender = copy_test.groupby("gender")

In [23]:
gender.count()["author"]

gender
NotKnown    5044
female       780
male        8578
Name: author, dtype: int64

In [24]:
test.to_csv('gender_mozilla.csv')